In [5]:
from keras.layers import Input, Dense , merge
from keras.layers.core import Reshape
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import Convolution2D,MaxPooling2D,UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adadelta, RMSprop
import os
import os.path
import numpy as np
import numpy 
from PIL import Image
from numpy import * 
# SKLEARN
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
import scipy.misc
import math
from scipy import signal

In [6]:
def comp(a,b):
	if(int(a[6:-4])>=int(b[6:-4])):
		return 1
	else:
		return -1

In [8]:
input_img = Input(shape=(240,320,3))  # adapt this if using `channels_first` image data format
x = Convolution2D(64, (3, 3), activation='relu', padding='same')(input_img)  # 160x180x16
x=BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x) # 80x90x16
x = Convolution2D(128, (3, 3), activation='relu', padding='same')(x)  
x=BatchNormalization()(x)
x = Convolution2D(128, (3, 3), activation='relu', padding='same')(x)  
x=BatchNormalization()(x)
x = Convolution2D(128, (3, 3), activation='relu', padding='same')(x)  
x=BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)   # 40x45x8
x = Convolution2D(256, (3, 3), activation='relu', padding='same')(x) 
x=BatchNormalization()(x)
x = Convolution2D(256, (3, 3), activation='relu', padding='same')(x) 
x=BatchNormalization()(x)
x = Convolution2D(256, (3, 3), activation='relu', padding='same')(x) 
x=BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)  #20x23x8
x = Convolution2D(512, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(512, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(512, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
encoded = MaxPooling2D((2, 2), padding='same',name='encoded')(x) 


# In[80]:


x=UpSampling2D((2,2))(encoded)
x = Convolution2D(512, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(512, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(512, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Convolution2D(256, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(256, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(256, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x=UpSampling2D((2,2))(x)
x = Convolution2D(128, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(128, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(128, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Convolution2D(64, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(64, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
x = Convolution2D(64, (3, 3), activation='relu', padding='same')(x)
x=BatchNormalization()(x)
decoded = Convolution2D(3, (3, 3), activation='sigmoid', padding='same')(x)
network=Model(input_img,decoded)
network.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 240, 320, 3)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 240, 320, 64)      1792      
_________________________________________________________________
batch_normalization_27 (Batc (None, 240, 320, 64)      256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 120, 160, 64)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 120, 160, 128)     73856     
_________________________________________________________________
batch_normalization_28 (Batc (None, 120, 160, 128)     512       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 120, 160, 128)     147584    
__________

In [81]:
rms=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.001)
network.compile(loss='mean_squared_error', optimizer=rms)

In [100]:
original_matrix=[]
transformed_matrix=[]

In [101]:
for k in range(1,2):
    path_major='Data/'+str(k)
    for j in range(1,3):
        im=array(Image.open(path_major+"/"+str(j)+".jpg"))
        img=im
        new=np.zeros([img.shape[0],img.shape[1],9])
        for i in range(3):
            new[:,:,i]=img[:,:,i]
        im=im.squeeze()#Remove acces dimension
        im=np.swapaxes(im,0,2)#Swap axes to feet thos of standart image (x,y,d)
        im=np.swapaxes(im,1,2)
        Gx=[[1,2,1], [0 , 0 ,0],[-1,-2,-1]]#Build sobel x,y gradient filters
        Gy=np.swapaxes(Gx,0,1)#Build sobel x,y gradient filters
        ndim=im[:,1,1].shape[0]# Get the depth (number of filter of the layer)
        TotGrad=np.zeros(im[1,:,:].shape) #The averge gradient map of the image to be filled later
        i=3
        for ii in range(ndim):# Go over all dimensions (filters) 
           gradx = signal.convolve2d(im[ii,:,:],Gx,  boundary='symm',mode='same');#Get x sobel response of ii layer
           grady = signal.convolve2d(im[ii,:,:],Gy,  boundary='symm',mode='same');#Get y sobel response of ii layer
           new[:,:,ii+i]+=gradx
           new[:,:,ii+i+1]+=grady
           i+=1
        original_matrix.append(new)
    for j in range(1,3):
        im=array(Image.open(path_major+"/"+str(j)+"_OF.jpg"))
        img=im
        new=np.zeros([img.shape[0],img.shape[1],9])
        for i in range(3):
            new[:,:,i]=img[:,:,i]
        im=im.squeeze()#Remove acces dimension
        im=np.swapaxes(im,0,2)#Swap axes to feet thos of standart image (x,y,d)
        im=np.swapaxes(im,1,2)
        Gx=[[1,2,1], [0 , 0 ,0],[-1,-2,-1]]#Build sobel x,y gradient filters
        Gy=np.swapaxes(Gx,0,1)#Build sobel x,y gradient filters
        ndim=im[:,1,1].shape[0]# Get the depth (number of filter of the layer)
        TotGrad=np.zeros(im[1,:,:].shape) #The averge gradient map of the image to be filled later
        i=3
        for ii in range(ndim):# Go over all dimensions (filters) 
           gradx = signal.convolve2d(im[ii,:,:],Gx,  boundary='symm',mode='same');#Get x sobel response of ii layer
           grady = signal.convolve2d(im[ii,:,:],Gy,  boundary='symm',mode='same');#Get y sobel response of ii layer
           new[:,:,ii+i]+=gradx
           new[:,:,ii+i+1]+=grady
           i+=1
        transformed_matrix.append(new)
print('Images are loaded')

Images are loaded


In [102]:
data,Label = shuffle(original_matrix,transformed_matrix, random_state=2)
X_train,X_test,Y_train,Y_test=train_test_split(data, Label, test_size=0.5, random_state=2)
X_train=array(X_train)
Y_train=array(Y_train)
X_test=array(X_test)
Y_test=array(Y_test)

In [103]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.
y_train = Y_train.astype('float32') / 255.
y_test = Y_test.astype('float32') / 255.
print('Trainset-'+ str(x_train.shape[0]))
print('Testset-' + str(x_test.shape[0]))

Trainset-1
Testset-1


In [112]:
for epoch in range(1):
						train_X,train_Y=shuffle(x_train,y_train)
						print ("Epoch is: %d\n" % epoch)
						batch_size=64
						num_batches=math.ceil(len(train_X)/batch_size)
						print ("Number of batches: %d\n" % int(num_batches))
						for batch in range(num_batches):
							batch_train_X=train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
							batch_train_Y=train_Y[batch*batch_size:min((batch+1)*batch_size,len(train_Y))]
							loss=network.train_on_batch(batch_train_X,batch_train_Y)
							print ('epoch_num: %d batch_num: %d train loss: %f\n' % (epoch,batch,loss))
						network.save_weights("model_encoder_withbatchnorm_deep_tt12.h5")
						if(epoch%1==0):
							x_test,y_test=shuffle(x_test,y_test)
							for batch in range(num_batches):
								batch_test_X=x_test[batch*batch_size:min((batch+1)*batch_size,len(x_test))]
								batch_test_Y=y_test[batch*batch_size:min((batch+1)*batch_size,len(y_test))]
								loss=network.test_on_batch(batch_test_X,batch_test_Y)
								print ('epoch_num: %d batch_num: %d test loss: %f\n' % (epoch,batch,loss))
							decoded_imgs=network.predict(x_test[:1])                            
							temp = np.zeros([240, 320*3,3])
							temp[:, :320,:1] = x_test[0,:,:,:1]
							temp[:, 320:320*2,:1] = y_test[0,:,:,:1]
							temp[:, 320*2:,:1] = decoded_imgs[0,:,:,:1]
							# temp[:,:,1]=temp[:,:,0]
							# temp[:,:,2]=temp[:,:,0]
							temp[:, :320,1:2] = x_test[0,:,:,1:2]
							temp[:, 320:320*2,1:2] = y_test[0,:,:,1:2]
							temp[:, 320*2:,1:2] = decoded_imgs[0,:,:,1:2]

							temp[:, :320,2:3] = x_test[0,:,:,2:3]
							temp[:, 320:320*2,2:3] = y_test[0,:,:,2:3]
							temp[:, 320*2:,2:3] = decoded_imgs[0,:,:,2:3]


							temp = temp*255
							scipy.misc.imsave('res/' + str(epoch) + ".jpg", temp)

						# if(epoch%10==0):
		
						# 	n = 10 # how many digits we will display
						# 	plt.figure(figsize=(3200, 720))
						# 	for i in range(n):
						# 	# display original
						# 		ax = plt.subplot(3, n, i + 1)
						# 		ax.get_xaxis().set_visible(False)
						# 		ax.get_yaxis().set_visible(False)
						# 		plt.imshow(x_test[i])
							 
						# 		ax = plt.subplot(3, n, i + 1+n)
						# 		ax.get_xaxis().set_visible(False)
						# 		ax.get_yaxis().set_visible(False)
						# 		plt.imshow(y_test[i])
							
						# 		ax = plt.subplot(3, n, i + 1 + n + n)
						# 		ax.get_xaxis().set_visible(False)
						# 		ax.get_yaxis().set_visible(False)
						# 		plt.imshow(decoded_imgs[i])
							
						# 	plt.savefig("results/" +  str(n_part) + "%#06d.jpg" % (epoch))
						# 	plt.close()

						del original_matrix[:]
						del transformed_matrix[:]


Epoch is: 0

Number of batches: 1

epoch_num: 0 batch_num: 0 train loss: 0.131458

epoch_num: 0 batch_num: 0 test loss: 0.262842



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [111]:
for i in range(1,2):
	conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
	conv1 = BatchNormalization()(conv1)
	conv1 = Conv2D(64, (3,3), activation='relu', padding='same')(conv1)
	conv1 = BatchNormalization()(conv1)
	conv1 = Conv2D(64, (3,3), activation='relu', padding='same')(conv1)
	conv1 = BatchNormalization()(conv1)
	pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
	conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
	conv2 = BatchNormalization()(conv2)
	conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
	conv2 = BatchNormalization()(conv2)
	conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
	conv2 = BatchNormalization()(conv2)
	pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
	conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
	conv3 = BatchNormalization()(conv3)
	conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
	conv3 = BatchNormalization()(conv3)
	conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
	conv3 = BatchNormalization()(conv3)
	pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
	conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
	conv4 = BatchNormalization()(conv4)
	conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
	conv4 = BatchNormalization()(conv4)
	conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
	conv4 = BatchNormalization()(conv4)
	pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
	conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
	conv5 = BatchNormalization()(conv5)
	conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
	conv5 = BatchNormalization()(conv5)
	conv5 = Conv2D(512, (3, 3), activation='sigmoid', padding='same')(conv5)
	conv5 = BatchNormalization()(conv5)


	up6 = UpSampling2D((2,2))(conv5)
	up6 = merge([up6, conv4], mode='concat', concat_axis=3)
	conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
	conv6 = BatchNormalization()(conv6)
	conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
	conv6 = BatchNormalization()(conv6)
	conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
	conv6 = BatchNormalization()(conv6)
	up7 = UpSampling2D((2,2))(conv6)
	up7 = merge([up7, conv3], mode='concat', concat_axis=3)
	conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
	conv7 = BatchNormalization()(conv7)
	conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
	conv7 = BatchNormalization()(conv7)
	conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
	conv7 = BatchNormalization()(conv7)
	up8 = UpSampling2D((2,2))(conv7)
	up8 = merge([up8, conv2], mode='concat', concat_axis=3)
	conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
	conv8 = BatchNormalization()(conv8)
	conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
	conv8 = BatchNormalization()(conv8)
	conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
	conv8 = BatchNormalization()(conv8)
	up9 = UpSampling2D((2,2))(conv8)
	up9 = merge([up9, conv1], mode='concat', concat_axis=3)
	conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
	conv9 = BatchNormalization()(conv9)
	conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
	conv9 = BatchNormalization()(conv9)	
	conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
	conv9 = BatchNormalization()(conv9)
	decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(conv9)
	new=Model(input_img,decoded)
	new.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/chucha/.local/lib/python3.6/site-packages/keras/legacy/layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarnin

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 240, 320, 9)  0                                            
__________________________________________________________________________________________________
conv2d_198 (Conv2D)             (None, 240, 320, 64) 5248        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_187 (BatchN (None, 240, 320, 64) 256         conv2d_198[0][0]                 
__________________________________________________________________________________________________
conv2d_199 (Conv2D)             (None, 240, 320, 64) 36928       batch_normalization_187[0][0]    
__________________________________________________________________________________________________
batch_norm